In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

!pip install optuna
import optuna

!pip install lightgbm
import lightgbm as lgb

In [2]:
PATH = "d:\\temp\\dphi\\50"

In [7]:
df_train = pd.read_csv(PATH + "/train_dataset.csv", index_col=0)
df_test = pd.read_csv(PATH + "/test_data.csv")

In [8]:
df_train

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,'no checking',18,'critical/other existing credit',radio/tv,1800,'<100','1<=X<4',4,'male single',none,...,car,24,none,own,2,skilled,1,none,yes,1
1,'<0',24,'existing paid',radio/tv,2439,'<100','<1',4,'female div/dep/mar',none,...,'real estate',35,none,own,1,skilled,1,yes,yes,0
2,'no checking',36,'no credits/all paid',repairs,2613,'<100','1<=X<4',4,'male single',none,...,car,27,none,own,2,skilled,1,none,yes,1
3,'no checking',15,'existing paid',education,4623,'100<=X<500','1<=X<4',3,'male single',none,...,'life insurance',40,none,own,1,'high qualif/self emp/mgmt',1,yes,yes,0
4,'<0',21,'existing paid',furniture/equipment,3599,'<100','4<=X<7',1,'female div/dep/mar',none,...,car,26,none,rent,1,'unskilled resident',1,none,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,'0<=X<200',60,'all paid',other,14782,'100<=X<500','>=7',3,'female div/dep/mar',none,...,'no known property',60,bank,'for free',2,'high qualif/self emp/mgmt',1,yes,yes,0
796,'0<=X<200',12,'critical/other existing credit','new car',2366,'500<=X<1000','4<=X<7',3,'male div/sep',none,...,car,36,none,own,1,'high qualif/self emp/mgmt',1,yes,yes,1
797,'no checking',9,'critical/other existing credit','new car',1224,'<100','1<=X<4',3,'male single',none,...,'real estate',30,none,own,2,skilled,1,none,yes,1
798,'<0',18,'no credits/all paid',furniture/equipment,3114,'<100','<1',1,'female div/dep/mar',none,...,'life insurance',26,none,rent,1,skilled,1,none,yes,0


In [9]:
for col in list(df_train.columns):
  print(df_train[col].value_counts())

'no checking'    308
'<0'             225
'0<=X<200'       216
'>=200'           51
Name: checking_status, dtype: int64
24    147
12    142
18     91
36     65
6      60
15     47
9      43
48     39
30     33
21     20
10     17
60     12
42     11
27     10
11      9
20      8
8       7
39      5
45      5
4       5
7       4
13      4
14      3
28      3
33      2
22      2
5       1
40      1
47      1
16      1
54      1
26      1
Name: duration, dtype: int64
'existing paid'                     412
'critical/other existing credit'    243
'delayed previously'                 74
'all paid'                           40
'no credits/all paid'                31
Name: credit_history, dtype: int64
radio/tv                215
'new car'               180
furniture/equipment     148
business                 84
'used car'               83
education                42
repairs                  18
other                    12
'domestic appliance'     10
retraining                8
Name: purpose, d

In [10]:
for col in list(df_test.columns):
  print(df_test[col].value_counts())

'no checking'    86
'0<=X<200'       53
'<0'             49
'>=200'          12
Name: checking_status, dtype: int64
24    37
12    37
18    22
36    18
15    17
6     15
10    11
21    10
48     9
30     7
9      6
27     3
16     1
72     1
33     1
7      1
4      1
60     1
54     1
14     1
Name: duration, dtype: int64
'existing paid'                     118
'critical/other existing credit'     50
'delayed previously'                 14
'no credits/all paid'                 9
'all paid'                            9
Name: credit_history, dtype: int64
radio/tv                65
'new car'               54
furniture/equipment     33
'used car'              20
business                13
education                8
repairs                  4
'domestic appliance'     2
retraining               1
Name: purpose, dtype: int64
2241     2
1216     2
2662     1
1300     1
14555    1
        ..
1514     1
3660     1
1295     1
3595     1
654      1
Name: credit_amount, Length: 198, dtype: int64
'

In [11]:
from sklearn import preprocessing
for col in list(df_train.select_dtypes(include=object)):
    le = preprocessing.LabelEncoder()
    le.fit(df_train[col].astype("str"))

    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    df_train[col] = df_train[col].apply(lambda x: le_dict.get(str(x), -1))
    df_test[col] = df_test[col].apply(lambda x: le_dict.get(str(x), -1))

In [14]:
x = df_train.drop("class", axis=1)
y = df_train["class"]

In [15]:
def objectiveRFC(trial):
    model = RandomForestClassifier(
        max_depth=trial.suggest_categorical("max_depth", [2, 3, None]),
        n_estimators=trial.suggest_int("n_estimators", 30, 500),
        max_features=trial.suggest_int("max_features", 1, 10),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 10),
        bootstrap = trial.suggest_categorical("bootstrap", [True, False]),
        criterion = trial.suggest_categorical("criterion", ["gini", "entropy"]),
        random_state = 114514)
    score = cross_val_score(model, x, y, n_jobs=-1, cv=3, scoring="accuracy").mean()
    return score

# https://qiita.com/maskot1977/items/d768b627f102b6d94ce1
study = optuna.create_study(direction="maximize")
study.optimize(objectiveRFC, timeout=60)


[I 2021-10-10 13:28:34,261] A new study created in memory with name: no-name-fc03041e-8869-48a2-8c3c-36b540a6db0a
[I 2021-10-10 13:28:36,331] Trial 0 finished with value: 0.7299663015215944 and parameters: {'max_depth': None, 'n_estimators': 434, 'max_features': 10, 'min_samples_split': 5, 'min_samples_leaf': 7, 'bootstrap': False, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7299663015215944.
[I 2021-10-10 13:28:37,468] Trial 1 finished with value: 0.7012568875371951 and parameters: {'max_depth': 2, 'n_estimators': 215, 'max_features': 4, 'min_samples_split': 8, 'min_samples_leaf': 2, 'bootstrap': True, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7299663015215944.
[I 2021-10-10 13:28:38,395] Trial 2 finished with value: 0.7050022058892549 and parameters: {'max_depth': 3, 'n_estimators': 50, 'max_features': 3, 'min_samples_split': 5, 'min_samples_leaf': 1, 'bootstrap': True, 'criterion': 'gini'}. Best is trial 0 with value: 0.7299663015215944.
[I 2021-10-10 13:28:3

In [37]:
x_train, x_eval, y_train, y_eval = train_test_split(x, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=114514,
                                                    stratify=y
                                                    )


def objectiveXGB(trial):
    model = xgb.XGBClassifier(
        objective="binary:logistic",
        max_depth=trial.suggest_int("max_depth", 2, 20),
        n_estimators=1000,
        subsample=trial.suggest_uniform("subsample", 0.5, 1.0),
        colsample_bytree=trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        learning_rate=trial.suggest_loguniform("learning_rate", 1e-8, 1.0),
        use_label_encoder=False
    )
    model.fit(x_train, y_train, eval_set=[(x_eval, y_eval)], early_stopping_rounds=100, verbose=False)
    pred = model.predict(x_eval)
    score = accuracy_score(y_eval, pred)
    return score

# https://qiita.com/maskot1977/items/d768b627f102b6d94ce1
study = optuna.create_study(direction="maximize")
study.optimize(objectiveXGB, timeout=60)

[I 2021-10-10 14:03:04,283] A new study created in memory with name: no-name-44354c37-c1e9-47b7-8c47-8c58ceeb2235
[I 2021-10-10 14:03:04,408] Trial 0 finished with value: 0.7 and parameters: {'max_depth': 16, 'subsample': 0.6732347916776262, 'colsample_bytree': 0.7121088032323768, 'learning_rate': 0.7063940707881985}. Best is trial 0 with value: 0.7.


[14:03:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:03:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-10-10 14:03:05,324] Trial 1 finished with value: 0.725 and parameters: {'max_depth': 4, 'subsample': 0.6807427045933329, 'colsample_bytree': 0.6690996757897405, 'learning_rate': 2.0481241067915856e-07}. Best is trial 1 with value: 0.725.


[14:03:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-10-10 14:03:05,541] Trial 2 finished with value: 0.71875 and parameters: {'max_depth': 3, 'subsample': 0.7800547530085726, 'colsample_bytree': 0.7725175900478345, 'learning_rate': 2.99600479356647e-08}. Best is trial 1 with value: 0.725.


...

[I 2021-10-10 14:04:03,698] Trial 107 finished with value: 0.73125 and parameters: {'max_depth': 5, 'subsample': 0.8130192535394244, 'colsample_bytree': 0.8398083785252806, 'learning_rate': 0.01960517492244729}. Best is trial 27 with value: 0.76875.
[I 2021-10-10 14:04:03,865] Trial 108 finished with value: 0.6875 and parameters: {'max_depth': 18, 'subsample': 0.992377626079918, 'colsample_bytree': 0.5374916758550805, 'learning_rate': 0.5713354237991379}. Best is trial 27 with value: 0.76875.


[14:04:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:04:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-10-10 14:04:04,132] Trial 109 finished with value: 0.75 and parameters: {'max_depth': 6, 'subsample': 0.8903352335027814, 'colsample_bytree': 0.9999222962624984, 'learning_rate': 0.04007079944879946}. Best is trial 27 with value: 0.76875.


[14:04:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-10-10 14:04:05,427] Trial 110 finished with value: 0.74375 and parameters: {'max_depth': 9, 'subsample': 0.8755693672392424, 'colsample_bytree': 0.7399458487428779, 'learning_rate': 0.0034599888269001363}. Best is trial 27 with value: 0.76875.


In [23]:
!pip install -U catboost

In [50]:

# https://qiita.com/shin_mura/items/8f1aa1ec90fa4ad6253e
from catboost import Pool, CatBoostClassifier

x_train, x_eval, y_train, y_eval = train_test_split(x, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=114514,
                                                    stratify=y
                                                    )
train_pool = Pool(x_train, y_train)
eval_pool = Pool(x_eval, y_eval)

def objectiveCB(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 50, 1000),
        depth=trial.suggest_int("depth", 4, 10),
        learning_rate=trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        random_strength=trial.suggest_int("random_strength", 0, 100),
        bagging_temperature=trial.suggest_loguniform("bagging_temperature", 0.01, 100.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50)
    )
    model.fit(train_pool, verbose=False)
    pred = model.predict(eval_pool)
    score = accuracy_score(y_eval, np.rint(pred))
    return score

# https://qiita.com/maskot1977/items/d768b627f102b6d94ce1
study = optuna.create_study(direction="maximize")
study.optimize(objectiveCB, timeout=600)

[I 2021-10-10 14:14:28,080] A new study created in memory with name: no-name-ab187696-648a-4ed2-996e-0e578fcdc763
[I 2021-10-10 14:14:29,406] Trial 0 finished with value: 0.75 and parameters: {'iterations': 722, 'depth': 7, 'learning_rate': 0.2604945621717957, 'random_strength': 29, 'bagging_temperature': 9.209546582055387, 'od_type': 'Iter', 'od_wait': 16}. Best is trial 0 with value: 0.75.
[I 2021-10-10 14:14:29,674] Trial 1 finished with value: 0.725 and parameters: {'iterations': 100, 'depth': 8, 'learning_rate': 0.07228099090933697, 'random_strength': 47, 'bagging_temperature': 49.98307725997545, 'od_type': 'Iter', 'od_wait': 21}. Best is trial 0 with value: 0.75.
[I 2021-10-10 14:14:36,540] Trial 2 finished with value: 0.74375 and parameters: {'iterations': 713, 'depth': 10, 'learning_rate': 0.2278248364169788, 'random_strength': 76, 'bagging_temperature': 66.88277875934926, 'od_type': 'IncToDec', 'od_wait': 39}. Best is trial 0 with value: 0.75.
[I 2021-10-10 14:14:37,347] Trial

In [46]:
def objectiveLGBM(trial):
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.1, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_child_weight":  trial.suggest_loguniform("min_child_weight", 0.5, 2),
        "min_child_samples":  trial.suggest_int("min_child_samples", 5, 20),
        "subsample": trial.suggest_loguniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-3, 1e3),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 1e3),
    }
    model = lgb.LGBMClassifier(
        verbose=-1,
        num_leaves=80,
        **params
    )
    score = cross_val_score(model, x, y, n_jobs=-1, cv=3, scoring="accuracy").mean()
    return score


study = optuna.create_study(direction="maximize")
study.optimize(objectiveLGBM, timeout=60)

[I 2021-10-10 14:09:22,412] A new study created in memory with name: no-name-ec1da53b-8649-4be2-a38d-1df64ec2566b
[I 2021-10-10 14:09:22,499] Trial 0 finished with value: 0.7287601025034496 and parameters: {'learning_rate': 0.10423428809454213, 'n_estimators': 712, 'max_depth': 5, 'min_child_weight': 0.6755080933252245, 'min_child_samples': 14, 'subsample': 0.8689767558293431, 'colsample_bytree': 0.8743167976008775, 'reg_lambda': 0.19726676955032166, 'reg_alpha': 1.5148366449283603}. Best is trial 0 with value: 0.7287601025034496.
[I 2021-10-10 14:09:22,781] Trial 1 finished with value: 0.7499835731651227 and parameters: {'learning_rate': 0.10811459092078256, 'n_estimators': 818, 'max_depth': 5, 'min_child_weight': 1.0954328764270922, 'min_child_samples': 10, 'subsample': 0.8343059697211955, 'colsample_bytree': 0.7096250556119362, 'reg_lambda': 5.89052702524057, 'reg_alpha': 0.032584447361625295}. Best is trial 1 with value: 0.7499835731651227.
[I 2021-10-10 14:09:22,865] Trial 2 finis

In [51]:
# model = RandomForestClassifier(
#     random_state=114514,
#     **study.best_params
# )

# model = xgb.XGBClassifier(
#     objective="binary:logistic",
#     n_estimators=1000,
#     use_label_encoder=False,
#     **study.best_params
# )


model = CatBoostClassifier(
    **study.best_params
)

# model = lgb.LGBMClassifier(
#     verbose=-1,
#     num_leaves=80,
#     **study.best_params
# )

# gacha

# CB: 20%/60s, parameter renewal: 85.81
# LGB: 60s parameter renewal: 85.42
# XGB: 600s: 84.83
# RFC: 60s: 84.31
# CB: 60s: 84
# XGB: 20%/60s: 83.95 ('A~)
# CB: 600s: 83.44 ('A`)
# LGB: 60s: 82.67

model.fit(x, y)
pred = model.predict(df_test)
pred = np.rint(pred).astype(int)

# model.fit(x, y)
# pred = model.predict(df_test)



0:	learn: 0.6575527	total: 10.4ms	remaining: 6.52s
1:	learn: 0.6309714	total: 20.7ms	remaining: 6.48s
2:	learn: 0.6024210	total: 30.4ms	remaining: 6.35s
3:	learn: 0.5681317	total: 39.9ms	remaining: 6.24s
4:	learn: 0.5431766	total: 45.3ms	remaining: 5.65s
5:	learn: 0.5245251	total: 55ms	remaining: 5.71s
6:	learn: 0.5205359	total: 55.8ms	remaining: 4.96s
7:	learn: 0.5037240	total: 65.2ms	remaining: 5.06s
8:	learn: 0.5000873	total: 66.2ms	remaining: 4.56s
9:	learn: 0.4962659	total: 67.4ms	remaining: 4.17s
10:	learn: 0.4859393	total: 68.3ms	remaining: 3.84s
11:	learn: 0.4726348	total: 73.4ms	remaining: 3.77s
12:	learn: 0.4536752	total: 82.6ms	remaining: 3.91s
13:	learn: 0.4441030	total: 92.3ms	remaining: 4.05s
14:	learn: 0.4388650	total: 97.7ms	remaining: 4s
15:	learn: 0.4233383	total: 108ms	remaining: 4.12s
16:	learn: 0.4156355	total: 117ms	remaining: 4.21s
17:	learn: 0.4121352	total: 119ms	remaining: 4.05s
18:	learn: 0.4065448	total: 129ms	remaining: 4.13s
19:	learn: 0.4006784	total: 138

In [52]:
pred

array([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1])

In [53]:
from datetime import datetime
save_to = PATH + "/submission_" + datetime.now().strftime("%Y%m%d-%H%M") + ".csv"
pd.DataFrame({"prediction": pred}).to_csv(save_to, index=False)
